In [3]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
from api_keys import gkey

In [4]:
weather_data = pd.read_csv('weather_data.csv')
weather_data.head()

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed
0,Porto-Novo,6.4965,2.6036,25.90,84,89,1.80
1,Yellowknife,62.4560,-114.3525,-29.77,82,40,5.66
2,Geraldton,-28.7667,114.6000,23.96,43,40,5.66
3,Iaciara,-14.0958,-46.6317,20.72,96,100,0.97
4,Hobart,-42.8794,147.3294,29.92,35,0,1.79


In [5]:
gmaps.configure(api_key=gkey)

In [6]:
locations = weather_data[["Lat", "Lng"]].astype(float)

In [7]:
humidity=weather_data['Humidity']

In [8]:
fig = gmaps.figure()

heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=100,
                                 point_radius = 1)

fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

In [95]:
#temperature lower than 30C
better_temperature = weather_data.loc[weather_data["Max Temp"]<30]
better_temperature.head()

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed
0,Porto-Novo,6.4965,2.6036,25.90,84,89,1.80
1,Yellowknife,62.4560,-114.3525,-29.77,82,40,5.66
2,Geraldton,-28.7667,114.6000,23.96,43,40,5.66
3,Iaciara,-14.0958,-46.6317,20.72,96,100,0.97
4,Hobart,-42.8794,147.3294,29.92,35,0,1.79


In [96]:
#and temperature higher than 20C
best_temperature = better_temperature.loc[better_temperature["Max Temp"]>20]
best_temperature.head()

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed
0,Porto-Novo,6.4965,2.6036,25.90,84,89,1.80
2,Geraldton,-28.7667,114.6000,23.96,43,40,5.66
3,Iaciara,-14.0958,-46.6317,20.72,96,100,0.97
4,Hobart,-42.8794,147.3294,29.92,35,0,1.79
9,Kahului,20.8947,-156.4700,26.01,79,20,3.60


In [97]:
#wind speed <10
perfect_wind = best_temperature.loc[best_temperature["Wind Speed"]<10.0]
perfect_wind.head()

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed
0,Porto-Novo,6.4965,2.6036,25.90,84,89,1.80
2,Geraldton,-28.7667,114.6000,23.96,43,40,5.66
3,Iaciara,-14.0958,-46.6317,20.72,96,100,0.97
4,Hobart,-42.8794,147.3294,29.92,35,0,1.79
9,Kahului,20.8947,-156.4700,26.01,79,20,3.60


In [98]:
#cloudiness=0
perfect_weather = perfect_wind.loc[perfect_wind["Cloudiness"]<1]
perfect_weather.head(15)

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed
4,Hobart,-42.8794,147.3294,29.92,35,0,1.79
34,Pisco,-13.7000,-76.2167,20.57,83,0,1.03
68,Champerico,14.3000,-91.9167,27.80,76,0,2.60
94,Salalah,17.0151,54.0924,21.05,60,0,3.09
95,Lampa,-33.2833,-70.9000,20.36,52,0,4.12
109,Mar del Plata,-38.0023,-57.5575,20.61,85,0,2.68
234,San Vicente,-26.6167,-54.1333,21.67,47,0,2.74
341,Matam,15.6175,-13.3303,24.00,43,0,1.03
388,Buenos Aires,-34.6132,-58.3772,23.31,80,0,8.75
416,Pakokku,21.3333,95.1000,21.06,55,0,2.89


In [99]:
hotel_df = pd.DataFrame(perfect_weather)
hotel_df.head()

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed
4,Hobart,-42.8794,147.3294,29.92,35,0,1.79
34,Pisco,-13.7000,-76.2167,20.57,83,0,1.03
68,Champerico,14.3000,-91.9167,27.80,76,0,2.60
94,Salalah,17.0151,54.0924,21.05,60,0,3.09
95,Lampa,-33.2833,-70.9000,20.36,52,0,4.12


In [100]:
hotel_df["Hotel Name"]=""
hotel_df.head()

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Hotel Name
4,Hobart,-42.8794,147.3294,29.92,35,0,1.79,
34,Pisco,-13.7000,-76.2167,20.57,83,0,1.03,
68,Champerico,14.3000,-91.9167,27.80,76,0,2.60,
94,Salalah,17.0151,54.0924,21.05,60,0,3.09,
95,Lampa,-33.2833,-70.9000,20.36,52,0,4.12,


In [105]:
params = {
    "radius": 5000,
    "types": "hotel",
    "keyword": "hotel",
    "key":gkey
}
for index, row in hotel_df.iterrows():
    lat = row["Lat"]
    lng = row["Lng"]
    
    params["location"] = f"{lat},{lng}"
    
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    
    name = requests.get(base_url, params=params)
    
    name = name.json()
    
    try:
        hotel_df.loc[index,'Hotel Name'] = name['results'][0]['name']
    
    except (KeyError, IndexError):
         print("Missing data")

Missing data


In [107]:
hotel_df.head(10)

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Hotel Name
4,Hobart,-42.8794,147.3294,29.92,35,0,1.79,Wrest Point
34,Pisco,-13.7000,-76.2167,20.57,83,0,1.03,Hotel San Jorge Residencial
68,Champerico,14.3000,-91.9167,27.80,76,0,2.60,Hotel Miramar
94,Salalah,17.0151,54.0924,21.05,60,0,3.09,Darbat Hotel
95,Lampa,-33.2833,-70.9000,20.36,52,0,4.12,
109,Mar del Plata,-38.0023,-57.5575,20.61,85,0,2.68,Hotel Amsterdam
234,San Vicente,-26.6167,-54.1333,21.67,47,0,2.74,Hotel Ceferino Rodriguez
341,Matam,15.6175,-13.3303,24.00,43,0,1.03,Hôtel Haayoo
388,Buenos Aires,-34.6132,-58.3772,23.31,80,0,8.75,Hilton Buenos Aires
416,Pakokku,21.3333,95.1000,21.06,55,0,2.89,Hotel Juno


In [114]:
hotel_name = hotel_df["Hotel Name"].tolist()

In [131]:
hotel_layer = gmaps.symbol_layer(
    locations, fill_color='rgba(0, 150, 0, 0.4)',
    stroke_color='rgba(0, 0, 150, 0.4)', scale=2,
    info_box_content=[f"{hotel}" for hotel in hotel_name]

)


fig = gmaps.figure()
fig.add_layer(hotel_layer)

fig

Figure(layout=FigureLayout(height='420px'))

In [129]:
fig = gmaps.figure()

fig.add_layer(heat_layer)
fig.add_layer(hotel_layer)

fig

Figure(layout=FigureLayout(height='420px'))